In [47]:
import numpy as np
from linearclassifier import *

## 1) Minimizing empirical risk

### `Expression of optimal theta*`

In [48]:
# th_opt = np.dot(np.linalg.inv(X @ X.T), X @ Y.T)

## 2) Adding reguralization 

### `X@X.T` is invertible?

In [49]:
X = np.array([[1, 2], [2, 3], [3, 5], [1, 4]])

In [50]:
X@X.T

array([[ 5,  8, 13,  9],
       [ 8, 13, 21, 14],
       [13, 21, 34, 23],
       [ 9, 14, 23, 17]])

#### We can see that row 1 and 2 sum to row 3. So X's rows are not linearly independant.

## 3) Linear regression going down

#### # In all the following definitions: x is d by n : input data y is 1 by n : output regression values th is d by 1 : weights th0 is 1 by 1 or scalar

In [51]:
def lin_reg(x, th, th0):
    return np.dot(th.T, x) + th0

def square_loss(x, y, th, th0):
    return (y - lin_reg(x, th, th0))**2

def mean_square_loss(x, y, th, th0):
     # the axis=1 and keepdims=True are important when x is a full matrix
     return np.mean(square_loss(x, y, th, th0), axis= 1, keepdims=True)
 


### `Data set `

In [52]:
#  x is d by n : input data
X = np.array([[1., 2., 3., 4.], [1., 1., 1., 1.]]) 
#  y is 1 by n : output regression values
Y = np.array([[1., 2.2, 2.8, 4.1]])
# th is d by 1 : weights
th = np.array([[1.0],[0.05]])
# th0 is 1 by 1 or scalar
th0 = np.array([[0.]])

### Gradient with respect to th

In [53]:
def d_lin_reg_th(x, th, th0):
    """RThis function returns the gradient of lin_reg with respect to th

    Args:
        x (dxn): input data , contains features on examples
        y (1xn): output regression values
        th (dx1): weights
        th0 (1x1): scalar, floats linear regression line to prevent overfiting
    """
    return x

def d_square_loss_th(x, y, th, th0):
    """This function returns the gradient of lin_square_loss with respect to th. It uses lin_reg and d_lin_reg"""
    return -2 * (y - lin_reg(x, th, th0)) * d_lin_reg_th(x, th, th0)


def d_mean_square_loss_th(x, y, th, th0):
    """This function returns the gradient of mean_square_loss with respect to th. It uses d_square_loss_th."""
    return np.mean(d_square_loss_th(x, y, th, th0), axis=1, keepdims=True)


### `Test cases`

#### `Test 1`

In [54]:
ans = d_lin_reg_th(X[:,0:1], th, th0)
ans

array([[1.],
       [1.]])

#### `Test 2`

In [55]:
ans = d_lin_reg_th(X, th, th0).tolist()
ans

[[1.0, 2.0, 3.0, 4.0], [1.0, 1.0, 1.0, 1.0]]

#### `Test 3`

In [56]:
d_square_loss_th(X[:, 0:1], Y[:, 0:1], th, th0).tolist()

[[0.10000000000000009], [0.10000000000000009]]

#### `Test 4`

In [57]:
d_square_loss_th(X, Y, th, th0).tolist()

[[0.10000000000000009, -0.6000000000000014, 1.5, -0.3999999999999986],
 [0.10000000000000009, -0.3000000000000007, 0.5, -0.09999999999999964]]

#### `Test 5`

In [58]:
d_mean_square_loss_th(X[:, 0:1], Y[:, 0:1], th, th0).tolist()

[[0.10000000000000009], [0.10000000000000009]]

#### `Test 6`

In [59]:
d_mean_square_loss_th(X, Y, th, th0).tolist()

[[0.15000000000000002], [0.04999999999999993]]

### Gradient with respect to th0

In [60]:
def d_lin_reg_th0(x, th, th0):
    """RThis function returns the gradient of lin_reg with respect to th0

    Args:
        x (dxn): input data , contains features on examples
        y (1xn): output regression values
        th (dx1): weights
        th0 (1x1): scalar, floats linear regression line to prevent overfiting
    """
    return np.ones((1, x.shape[1]))

def d_square_loss_th0(x, y, th, th0):
    """This function returns the gradient of lin_square_loss with respect to th0. It uses lin_reg and d_lin_reg_th0"""
    return -2 * (y - lin_reg(x, th, th0)) * d_lin_reg_th0(x, th, th0)


def d_mean_square_loss_th0(x, y, th, th0):
    """This function returns the gradient of mean_square_loss with respect to th. It uses d_square_loss_th0."""
    return np.mean(d_square_loss_th0(x, y, th, th0), axis=1, keepdims=True)

#### `Test case 1`

In [61]:
d_lin_reg_th0(X[:, 0:1], th, th0).tolist()

[[1.0]]

#### `Test 2`

In [62]:
d_lin_reg_th0(X, th, th0).tolist()

[[1.0, 1.0, 1.0, 1.0]]

#### `Test case 3`

In [63]:
d_square_loss_th0(X[:, 0:1], Y[:, 0:1], th, th0).tolist()

[[0.10000000000000009]]

#### `Test case 4`

In [64]:
d_square_loss_th0(X, Y, th, th0).tolist()

[[0.10000000000000009, -0.3000000000000007, 0.5, -0.09999999999999964]]

#### `Test case 5`

In [65]:
d_mean_square_loss_th0(X[:, 0:1], Y[:, 0:1], th, th0).tolist()

[[0.10000000000000009]]

#### `Test case 6`

In [66]:
d_mean_square_loss_th0(X, Y, th, th0).tolist()

[[0.04999999999999993]]

## 4) Going down the rigde

### Description

In [67]:
# In all the following definitions:
# x is d by n : input data
# y is 1 by n : output regression values
# th is d by 1 : weights
# th0 is 1 by 1 or scalar

### Rigde Objective function

In [68]:
def ridge_obj(x, y, th, th0, lam):
    return mean_square_loss(x, y, th, th0) + lam * np.linalg.norm(th)**2

### Gradient og rigde objective function with respect to th and th0

In [69]:
def d_ridge_obj_th(x, y, th, th0, lam):
    return d_mean_square_loss_th(x, y, th, th0) + 2*lam*th

def d_ridge_obj_th0(x, y, th, th0, lam):
    return d_mean_square_loss_th0(x, y, th, th0)


### Test cases

#### `Test 1`

In [70]:
d_ridge_obj_th(X[:, 0:1], Y[:, 0:1], th, th0, 0.01).tolist()

[[0.12000000000000009], [0.10100000000000009]]

#### `Test 2`

In [71]:
d_ridge_obj_th(X, Y, th, th0, 0.05).tolist()

[[0.25], [0.05499999999999994]]

#### `Test 3`

In [72]:
d_ridge_obj_th0(X[:, 0:1], Y[:, 0:1], th, th0, 0.01).tolist()

[[0.10000000000000009]]

#### `Test 4`

In [73]:
d_ridge_obj_th0(X, Y, th, th0, 0.05).tolist()

[[0.04999999999999993]]

## 5) Stochastic gradient descent (sgd)

#### **We will implement `stochastic gradient descent` in general way as we did for gradient descent. Note that `stochastic` part refers to using a randomly selected point and corresponding label from the given dataset to perform an update.**

In [84]:
def num_grad(f, delta=0.001):
    def df(x):
        g = np.zeros(x.shape)
        for i in range(x.shape[0]):
            xi = x[i, 0]
            x[i, 0] = xi - delta
            xm = f(x)
            x[i, 0] = xi + delta
            xp = f(x)
            x[i, 0] = xi
            g[i, 0] = (xp - xm) / (2*delta)
        return g
    return df

In [75]:
def J(Xi, yi, w):
     # translate from (1-augmented X, y, theta) to (separated X, y, th, th0) format
     return float(ridge_obj(Xi[:-1, :], yi, w[:-1, :], w[-1:, :], 0))


In [76]:
def dJ(Xi, yi, w):
     def f(w):
         return J(Xi, yi, w)
     
     return num_grad(f)(w)

In [77]:
def sgd(X, y, J, dJ, w0, step_size_fn, max_iter):
    n = y.shape[1]
    prev_w = w0
    ws = []
    fs = []
    np.random.seed(0)
    for i in range(max_iter):
        j = np.random.randint(n)
        Xj = X[:, j:j+1]
        yj = y[:, j:j+1]
        prev_f = J(Xj, yj, prev_w)
        prev_grad = dJ(Xj, yj, prev_w)
        fs.append(prev_f)
        ws.append(prev_w)
        if i == max_iter - 1:
            return prev_w, fs, ws
        step = step_size_fn(i)
        prev_w = prev_w - step * prev_grad

### Datasets

In [78]:
def downwards_line():
    X = np.array([[0.0, 0.1, 0.2, 0.3, 0.42, 0.52, 0.72, 0.78, 0.84, 1.0],
                  [1.0, 1.0, 1.0, 1.0, 1.0,  1.0,  1.0,  1.0,  1.0,  1.0]])
    y = np.array([[0.4, 0.6, 1.2, 0.1, 0.22, -0.6, -1.5, -0.5, -0.5, 0.0]])
    return X, y

X, y = downwards_line()

In [83]:
X[:, 0:1].shape
#y[:, 0:1].shape
theta = cv([0.0, 0.0])
J(X, y, theta)

0.51284

### Test cases 

### Test function

In [80]:
def package_ans(gd_vals):
    x, fs, xs = gd_vals
    return [x.tolist(), [fs[0], fs[-1]], [xs[0].tolist(), xs[-1].tolist()]]

#### `Test 1`

In [85]:
package_ans(sgd(X, y, J, dJ, cv([0.0, 0.0]), lambda i: 0.1, 1000))

[[[-1.4118594928102899], [0.7705243986321614]],
 [0.36, 0.028653685126009333],
 [[[0.0], [0.0]], [[-1.4118594928102899], [0.7705243986321614]]]]

#### `Test 2`

In [86]:
package_ans(sgd(X, y, J, dJ, cv([-0.1, 0.1]), lambda i: 0.01, 1000))

[[[-1.168945161526412], [0.5656868407680322]],
 [0.419904, 0.023688169520937184],
 [[[-0.1], [0.1]], [[-1.168945161526412], [0.5656868407680322]]]]